### Import modules

In [1]:
import pandas as pd
import numpy as np

print(pd.__version__)
print(np.__version__)

1.2.3
1.20.1


### Membaca data set 

In [2]:
df = pd.read_excel('./dataset/laporan_iklim_harian_1.xlsx')
df.head(10)

,Tanggal,Tavg,RH_avg,ss,ff_avg,RR
0,01-01-2015,25.3,88.0,0.0,4.0,23.9
1,02-01-2015,26.3,85.0,0.0,4.0,10.5
2,03-01-2015,25.8,90.0,1.9,4.0,9.7
3,04-01-2015,NaN,NaN,NaN,NaN,NaN
4,05-01-2015,26.9,81.0,0.5,5.0,5.9
5,06-01-2015,27.5,78.0,8.2,5.0,0.0
6,07-01-2015,28.0,79.0,4.5,5.0,0.0
7,08-01-2015,NaN,NaN,NaN,NaN,NaN
8,09-01-2015,27.9,78.0,4.4,6.0,0.0
9,10-01-2015,NaN,NaN,NaN,NaN,NaN


### Ukuran datatset

In [3]:
df.shape # bentuk/dimensi dataset (baris,kolom)

(2246, 6)

### Mengganti nama kolom
    Tavg  : Temperatur rata-rata (°C) --> temperatur 
    RH_avg: Kelembapan rata-rata (%) --> kelembapan 
    ss    : Lamanya penyinaran matahari (jam) --> lama_penyinaran 
    ff_avg: Kecepatan angin rata-rata (m/s) --> kecepatan_angin 
    RR    : Curah hujan (mm) --> curah_hujan

In [4]:
df = df.rename(columns={'Tanggal':'tanggal','Tavg':'temperatur', 'RH_avg':'kelembapan',
                        'ss':'lama_penyinaran','ff_avg':'kecepatan_angin', 'RR':'curah_hujan'})
df.head()

,tanggal,temperatur,kelembapan,lama_penyinaran,kecepatan_angin,curah_hujan
0,01-01-2015,25.3,88.0,0.0,4.0,23.9
1,02-01-2015,26.3,85.0,0.0,4.0,10.5
2,03-01-2015,25.8,90.0,1.9,4.0,9.7
3,04-01-2015,NaN,NaN,NaN,NaN,NaN
4,05-01-2015,26.9,81.0,0.5,5.0,5.9


### Mengenali apakah ada missing values

In [5]:
df.isna().values.any() # mendeteksi keberadaan nilai kosong

True

### Menghapus baris yang mengandung missing values

In [6]:
df = df.dropna(axis='rows')
df.shape

(1944, 6)

In [7]:
df.isna().sum()

tanggal            0
temperatur         0
kelembapan         0
lama_penyinaran    0
kecepatan_angin    0
curah_hujan        0
dtype: int64

### Menghapus row di kolom curah hujan yang mengandung 8888 dan 9999
8888 : Data tidak terukur <br/>
9999 : Tidak Ada Data (tidak dilakukan pengukuran)

In [8]:
df = df[(df['curah_hujan'] != 8888) & (df['curah_hujan'] != 9999)]
df.shape

(1858, 6)

### Mengenali duplicat baris

In [9]:
df.duplicated().value_counts() # hitung jumlah duplikasi data

False    1858
dtype: int64

### Mereset ulang index dari nol

In [10]:
df = df.reset_index(drop=True)
df

,tanggal,temperatur,kelembapan,lama_penyinaran,kecepatan_angin,curah_hujan
0,01-01-2015,25.3,88.0,0.0,4.0,23.9
1,02-01-2015,26.3,85.0,0.0,4.0,10.5
2,03-01-2015,25.8,90.0,1.9,4.0,9.7
3,05-01-2015,26.9,81.0,0.5,5.0,5.9
4,06-01-2015,27.5,78.0,8.2,5.0,0.0
...,...,...,...,...,...,...
1853,21-03-2021,28.2,78.0,4.9,3.0,1.6
1854,22-03-2021,28.5,76.0,8.0,3.0,0.0
1855,23-03-2021,27.5,79.0,10.1,3.0,0.0
1856,24-03-2021,27.8,74.0,8.6,4.0,0.0


### Menambahkan kolom status dari kategori curah_hujan
sumber : https://www.bmkg.go.id/cuaca/probabilistik-curah-hujan.bmkg

In [11]:
df['status'] = pd.cut(df['curah_hujan'],
                      bins = [0, 0.5, 20, 50, 100, 150],
                      labels=['berawan', 'hujan ringan', 'hujan sedang',
                              'hujan lebat', 'hujan sangat lebat'],
                      include_lowest=True)

### Mengubah kolom status menjadi tipe data ordinal

In [12]:
from pandas.api.types import CategoricalDtype

tingkat_curah_hujan = CategoricalDtype(['berawan', 'hujan ringan', 'hujan sedang',
                                        'hujan lebat', 'hujan sangat lebat'], 
                                    ordered=True)

df['status'] = df['status'].astype(tingkat_curah_hujan)
df.dtypes

tanggal              object
temperatur          float64
kelembapan          float64
lama_penyinaran     float64
kecepatan_angin     float64
curah_hujan         float64
status             category
dtype: object

### Menghapus kolom curah_hujan
Karena kolom status sudah mewakilkan kolom curah_hujan, maka kolom curah_hujan tidak diperlukan

In [13]:
df.drop(columns='curah_hujan', inplace=True)
df.sort_values('status') # Tampilkan berdasarkan urutan status

,tanggal,temperatur,kelembapan,lama_penyinaran,kecepatan_angin,status
1857,25-03-2021,27.6,78.0,6.1,3.0,berawan
884,09-01-2018,27.3,80.0,3.8,2.0,berawan
882,07-01-2018,27.4,81.0,1.6,3.0,berawan
881,06-01-2018,27.7,74.0,1.5,5.0,berawan
880,05-01-2018,27.7,70.0,5.4,5.0,berawan
...,...,...,...,...,...,...
1510,01-02-2020,25.5,92.0,4.0,2.0,hujan sangat lebat
705,14-06-2017,26.7,83.0,0.0,2.0,hujan sangat lebat
1482,01-01-2020,26.7,88.0,0.5,2.0,hujan sangat lebat
451,18-06-2016,24.7,89.0,2.2,2.0,hujan sangat lebat


### Mengeksport dataframe menjadi excel dan menyimpannya

In [14]:
from pandas import ExcelWriter
from pandas import ExcelFile

In [15]:
writer = pd.ExcelWriter('./dataset/3KA32_laporan_iklim_harian_baru.xlsx')
df.to_excel(writer, index = False, sheet_name='Sheet1')
writer.save()